# Problem

Let $I = \{1, \ldots, n\}$a set of financial products. For every $ i \ in I $ are given:
- $r_i \in\mathbb{R}$ expected return in the next period;
$q_i \in\mathbb{Z}$ equivalent value of the security in the current portfolio;<-->
- $s_i \in S$ type of assets;
$\delta_i$ maximum variation in the invested value allowed with respect to the current value<-->
- $\phi^{max}_k$, $\phi^{min}_k \in [0,1]$ maximum and minimum fraction, respectively, of budget allocable on a single product $k, \forall k \in S$;
- $\gamma^{max}_i$, $\gamma^{min}_i \in [0,1]$ maximum and minimum ration, respectively, of budget allocable on a single product $i, \forall i \in I$.


**Product types**
The $S$ set contains possible types, for example, $S={A,O,M}$, with $A$ equity, $O$ bond, $M$ mixed. 

**Risk** The risk associated with the portfolio is modeled with the *parametric model*, i.e. through an assigned $Q$ covariance matrix and a maximum risk $R_{max}$ associated with the investor profile.

**Budget** Let's be $B$ the budget. This can be equal to the current $\sum_i^n{q_i}$ value, or specified otherwise, if you decide to invest new capital or disinvest.
Let this be the percentage countervalue of the single security, I define deltai as the maximum percentage variation.


# Optimization model
We define the decision variables (real)  $x_i$, $i \in I$, representing the percentage of value to be allocated on the $i$ product. The model is as follows:

\begin{align}
\label{eq:obj}  \max & \sum_{i = 1}^n r_i x_i & \nonumber \\
 \nonumber \mbox{s.t. } &\\
 & \sum_{i \in I} x_i \le 1, & \\
  & x^T Q x \leq R_{max}, & \\
  %& |x_i - q_i| \le \delta_i q_i, & i \in I \\
  & \sum_{i:s_i = k} x_i \ge \phi^{min}_k, & k \in S \\
  & \sum_{i:s_i = k} x_i \le \phi^{max}_k, & k \in S \\
  & x_i \ge \gamma^{min}_i, & i \in I \\
  & x_i \le \gamma^{max}_i, & i \in I \nonumber
\end{align}

In [23]:
#import the libraries
import pandas as pd
import numpy as np
import gurobipy as gb

In [24]:
# Take a matrix x as input and returns True if x is positive semidefinite, False otherwise

def is_pos_semidef(x):
    return np.all(np.linalg.eigvals(x) >= 0)

In [25]:
df_cov = pd.read_excel('DB_INDEX_DATA-01.xlsx', sheet_name='ALLOCATION-01-LCY-COVAR', skiprows=28, nrows=19)

In [26]:
# Drop unnecessary columns (N.B. depends on the actual sheet shape)
df_cov.drop(df_cov.columns[1], axis=1, inplace=True)

In [27]:
df_cov.set_index('Index\Index', inplace=True)

In [28]:
is_pos_semidef(df_cov.to_numpy())

True

In [29]:
# Import the excel with the predictions

df_allocation_postcumulation = pd.read_excel('prova-PREDICTED-POSTCUMULATION-ALLOCATION-01-LCY-REND.xlsx', index_col=0)

In [30]:
# Selection of columns to exlude from the dataset

matching = []

for macroclass in ['LIQUIDITY','EQUITY','BOND']:
    matching = matching + [column for column in df_allocation_postcumulation.columns.tolist() if macroclass in column]

print(matching)

['LIQUIDITY mean', 'LIQUIDITY max', 'LIQUIDITY min', 'LIQUIDITY var', 'EQUITY mean', 'EQUITY max', 'EQUITY min', 'EQUITY var', 'BOND mean', 'BOND max', 'BOND min', 'BOND var']


In [31]:
# Drop unnecessary columns

df_allocation_postcumulation.drop(columns=matching, inplace=True)

In [32]:
# Import the excel sheet containing 'Macro-Asset class'/'index' matchings

df_asset_class = pd.read_excel('DB_INDEX_DATA-01.xlsx', sheet_name='ALLOCATION-01', skiprows=[0])

In [33]:
# Forward fill of nan values 

df_asset_class.fillna(method='ffill', inplace=True)

In [34]:
# Extraction of the {Index : Macro-Asset Class} dictionary

d = df_asset_class[['Macro-Asset Class','Index Id']].set_index('Index Id').transpose().to_dict('records')
macroclasses = d[0]

In [35]:
# Rename dataframe columns

df_allocation_postcumulation.rename(dict(zip(df_allocation_postcumulation.columns,list(macroclasses.keys()))), axis='columns', inplace=True)

In [36]:
# Rename covariance matrix columns

df_cov.rename(dict(zip(df_cov.columns,list(macroclasses.keys()))), axis='columns', inplace=True)
df_cov.rename(dict(zip(df_cov.index,list(macroclasses.keys()))), axis='index', inplace=True)

In [58]:
# Set the confidence interval

conf_interval = 2.33

# Set the number of days in a trimester

horizon_days = 65

In [59]:
# Set the monetary budget to allocate

monetary_budget = 10000 

# Set maximum Value at Risk ([0.045 --> Conservativo] [0.095 --> Moderato] [0.25 --> Dinamico])

max_VaR = 0.095

# Set [min,max] bounds for each Macro-Asset Class

macroAssetClass, minBudget, maxBudget = gb.multidict({
    'Liquidity' : [0.05, 1],
    'Bond' : [0, 1],
    'Equity' : [0, 0.1]
})

# Set [min,max] bounds for each single asset

assets, minAssetBudget, maxAssetBudget = gb.multidict({
    'MXEM': [0, 1],
    'MXEU': [0, 1],
    'MXNA': [0, 1],
    'MXEF': [0, 1],
    'MXJP': [0, 1],
    'MXPC': [0, 1],
    'JPMGEMLC': [0, 1],
    'JNUCUK': [0, 1],
    'SBF14T': [0, 1],
    'ER00': [0, 1],
    'UC00': [0, 1],
    'JNUCUS': [0, 1],
    'C0A0': [0, 1],
    'JPEGCOMP': [0, 1],
    'JNUCJP': [0, 1],
    'JC00': [0, 1],
    'JPCAEU3M': [0, 1],
    'JPCAGB3M': [0, 1],
    'JPCAUS3M': [0, 1]
})

In [60]:
# Create the model

m = gb.Model("Portfolio_Opt")

In [61]:
# Create variables

x = m.addVars(list(macroclasses.keys()),
              obj = df_allocation_postcumulation.tail(1).to_numpy().tolist()[0], 
              name = 'x'
             )
m.update()

In [62]:
#Maximize function
m.ModelSense = gb.GRB.MAXIMIZE

In [63]:
# Add budget constraint

m.addConstr(x.sum() <= 1, name='budget')
m.update()

In [64]:
# Add min allocable budget per macroclass constraints

m.addConstrs((x.sum([k for k,v in macroclasses.items() if v == macroclass]) >= minBudget[macroclass] for macroclass in macroAssetClass), name="min_macroclass_budget")
m.update()

In [65]:
# Add max allocable budget per macroclass constraints

m.addConstrs((x.sum([k for k,v in macroclasses.items() if v == macroclass]) <= maxBudget[macroclass] for macroclass in macroAssetClass), name="max_macroclass_budget")
m.update()

In [66]:
# Add min allocable budget for each single asset constraints

m.addConstrs((x[k] >= v for k,v in minAssetBudget.items()), name="min_asset_budget")
m.update()

In [67]:
# Add max allocable budget for each single asset constraints

m.addConstrs((x[k] <= v for k,v in maxAssetBudget.items()), name="max_asset_budget")
m.update()

In [68]:
# Compute xQ

weighted_cov = {}
for k,v in df_cov.to_dict().items():
    weighted_cov[k] = x.prod(v) 

In [69]:
# Compute R_max
def rmax(horizon_days, conf_interval, max_VaR):
    r_max = (max_VaR/conf_interval)**2/horizon_days
    return r_max
    

In [70]:
r_max = rmax(horizon_days, conf_interval, max_VaR)

In [71]:
# Compute (xQ)*x'

m.addQConstr(x.prod(weighted_cov) <= r_max, "max_VaR")
m.update()

In [72]:
# Write the lp file containing the model

m.write('model.lp')

In [73]:
# Optimize the model

m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 45 rows, 19 columns and 95 nonzeros
Model fingerprint: 0x081e37e9
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-10, 2e-04]
  Objective range  [4e-04, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
  QRHS range       [3e-05, 3e-05]
Presolve removed 41 rows and 0 columns
Presolve time: 0.01s
Presolved: 23 rows, 39 columns, 232 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.260e+02
 Factor NZ  : 2.760e+02
 Factor Ops : 4.324e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.30224906e-02  9.50000000e-02  6.86e+00 1.00e-01  1.86e-02     0s
   1   4.80884545e-03  9.30868271e-02  1.67e-01 1.10e-07  1.71e-03     0s


In [74]:
# Get the solution

solution = m.getAttr('x', x) 

In [75]:
# Print out the percentages to allocate for each asset (rounded to the 2nd decimal)

for k, v in solution.items():
    print(k+': '+ str(round(v, 2)))

MXEM: 0.0
MXEU: 0.0
MXNA: 0.0
MXEF: 0.0
MXJP: 0.0
MXPC: 0.0
JPMGEMLC: 0.0
JNUCUK: 0.0
SBF14T: 0.0
ER00: 0.0
UC00: 0.0
JNUCUS: 0.95
C0A0: 0.0
JPEGCOMP: 0.0
JNUCJP: 0.0
JC00: 0.0
JPCAEU3M: 0.0
JPCAGB3M: 0.0
JPCAUS3M: 0.05


In [76]:
# Percentage of budget to allocate according to the optimal solution

sum(solution.values())

0.9999999997254649

In [77]:
# Monetary amount to allocate for each asset

total = 0
for k, v in solution.items():
    print(k+': '+ str(round(v*monetary_budget)))
    total += round(v*monetary_budget)
print("\n")
print('Total allocation: ' + str(total))

MXEM: 0
MXEU: 0
MXNA: 0
MXEF: 0
MXJP: 0
MXPC: 0
JPMGEMLC: 0
JNUCUK: 0
SBF14T: 0
ER00: 0
UC00: 0
JNUCUS: 9500
C0A0: 0
JPEGCOMP: 0
JNUCJP: 0
JC00: 0
JPCAEU3M: 0
JPCAGB3M: 0
JPCAUS3M: 500


Total allocation: 10000


In [78]:
# Print out the value of the objective function

m.getObjective().getValue()

0.0330520302722588